In [126]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
rec = pd.read_csv('recruitment.csv')
rec

,recruitment_seq,address_seq1,address_seq2,address_seq3,career_end,career_start,check_box_keyword,education,major_task,qualifications,text_keyword
0,R02264,3.0,NaN,NaN,0,0,2507;2707;2810,4,8,1,NaN
1,R06317,3.0,NaN,NaN,0,0,2204;2205;2707,3,2,1,NaN
2,R04017,3.0,NaN,NaN,0,0,2101;2108;2201;2707,3,2,1,NaN
3,R02865,3.0,NaN,NaN,0,0,2201;2204;2205;2707,2,2,1,NaN
4,R04890,3.0,NaN,NaN,0,0,2201;2204;2205;2707,2,2,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6690,R03678,3.0,NaN,NaN,0,0,2101;2108;2201;2204;2205;2707,3,2,1,NaN
6691,R04593,3.0,NaN,NaN,0,0,2201;2204;2205;2707,4,2,1,NaN
6692,R03252,3.0,NaN,NaN,0,0,2109,3,2,1,NaN
6693,R05130,3.0,NaN,NaN,0,0,2201;2204;2205;2707,2,2,2,NaN


In [127]:
res = pd.read_csv('resume.csv')
res2 = res.set_index(['resume_seq'])
res2

,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code
resume_seq,,,,,,,,,,,,
U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,재료·화학·섬유·의복,NaN,NaN,67,NaN
U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,재료·화학·섬유·의복,NaN,NaN,84,섬유;봉제;가방;의류
U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,니트디자인,재료·화학·섬유·의복,NaN,NaN,121,학교;학원;직원훈련(교육서비스)
U04599,2020-05-13,2020-05-28,4,2012,0.0,2500.0,MD;기획MD,재료·화학·섬유·의복,NaN,NaN,24,섬유;봉제;가방;의류
U07573,2019-07-23,2020-03-08,4,2010,1900.0,0.0,디자이너;남성복;스포츠웨어;편집디자인;코디네이터;일러스트레이터;VMD;MD,재료·화학·섬유·의복,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
U04890,2018-05-27,2020-08-26,5,1994,0.0,6250.0,상품기획;생산;글로벌소싱;남성복;여성복,재료·화학·섬유·의복,NaN,NaN,204,섬유;봉제;가방;의류
U05910,2019-09-27,2020-10-22,4,2013,2300.0,1900.0,소재;기획MD;남성복;온라인MD;디자이너,재료·화학·섬유·의복,NaN,NaN,12,섬유;봉제;가방;의류
U01468,2019-02-26,2020-06-23,4,2008,0.0,0.0,상품기획MD;우븐디자인,디자인,NaN,NaN,0,섬유;봉제;가방;의류


In [128]:
res_cer = pd.read_csv("resume_certificate.csv")
len(res_cer['resume_seq'].unique())
# 자격증 소유자 5976명

5976

In [129]:
len(res_cer['certificate_contents'].unique())

1787

In [130]:
for i in range(len(res_cer)):
    j = res_cer['certificate_contents'][i]
    if isinstance(j, str):  # 문자열 여부 확인
        if re.search("면허", j) is not None:
            if re.search("2종", j) is not None:
                res_cer['certificate_contents'][i] = '2종운전면허'
            elif re.search("1종", j) is not None:
                if re.search("대형", j) is not None:
                    res_cer['certificate_contents'][i] = '1종운전면허대형'
                elif re.search("보통", j) is not None:
                    res_cer['certificate_contents'][i] = '1종운전면허보통'
                else:
                    res_cer['certificate_contents'][i] = '1종운전면허보통'
        elif re.search("운전", j) is not None:
            if re.search("자동차", j) is not None:
                res_cer['certificate_contents'][i] = '2종운전면허'

In [143]:
for i in range(len(res_cer)):
    j = res_cer['certificate_contents'][i]
    if isinstance(j, str):  # 문자열 여부 확인
        if re.search("MOS", j) is not None:
            res_cer['certificate_contents'][i] = 'MOS'
        elif re.search("중국", j) is not None:
            res_cer['certificate_contents'][i] = '중국어'
        elif re.search("POWER", j) is not None:
            res_cer['certificate_contents'][i] = '파워포인트'
        elif re.search("EXCEL", j) is not None:
            res_cer['certificate_contents'][i] = '엑셀'
        elif re.search("엑셀", j) is not None:
            res_cer['certificate_contents'][i] = '엑셀'
        elif re.search("태권도", j) is not None:
            res_cer['certificate_contents'][i] = '태권도'
        elif re.search("합기도", j) is not None:
            res_cer['certificate_contents'][i] = '합기도'
        elif re.search("워드", j) is not None:
            res_cer['certificate_contents'][i] = '워드'
        elif re.search("컴활", j) is not None:
            res_cer['certificate_contents'][i] = '컴활'
        elif re.search("컴퓨터활용", j) is not None:
            res_cer['certificate_contents'][i] = '컴활'
        elif re.search("일본", j) is not None:
            res_cer['certificate_contents'][i] = '일본어'
        elif re.search("ITQ", j) is not None:
            res_cer['certificate_contents'][i] = 'ITQ'
        elif re.search("MOUS", j) is not None:
            res_cer['certificate_contents'][i] = 'MOUS'
        elif re.search("GTQ", j) is not None:
            res_cer['certificate_contents'][i] = 'GTQ'
        elif re.search("PHOTOSHOP", j) is not None:
            res_cer['certificate_contents'][i] = 'GTQ'
        elif re.search("WORD", j) is not None:
            res_cer['certificate_contents'][i] = '워드'
        elif re.search("패션", j) is not None:
            res_cer['certificate_contents'][i] = '패션'
        elif re.search("FASHION", j) is not None:
            res_cer['certificate_contents'][i] = '패션'
        elif re.search("교사", j) is not None:
            res_cer['certificate_contents'][i] = '교사'
        elif re.search("그래픽스", j) is not None:
            res_cer['certificate_contents'][i] = '컴퓨터그래픽스'
        elif re.search("스쿠버", j) is not None:
            res_cer['certificate_contents'][i] = '스쿠버'
        elif re.search("다이버", j) is not None:
            res_cer['certificate_contents'][i] = '스쿠버'
        elif re.search("영어", j) is not None:
            res_cer['certificate_contents'][i] = '영어'
        elif re.search("OPIC", j) is not None:
            res_cer['certificate_contents'][i] = '영어'
        elif re.search("ENGLISH", j) is not None:
            res_cer['certificate_contents'][i] = '영어'
        elif re.search("컬러리스트", j) is not None:
            res_cer['certificate_contents'][i] = '컬러리스트'
        elif re.search("한자", j) is not None:
            res_cer['certificate_contents'][i] = '한자'
        elif re.search("정보", j) is not None:
            res_cer['certificate_contents'][i] = '정보'
        elif re.search("사무자동화", j) is not None:
            res_cer['certificate_contents'][i] = '사무자동화'
        elif re.search("기획", j) is not None:
            res_cer['certificate_contents'][i] = '기획'
        elif re.search("체육", j) is not None:
            res_cer['certificate_contents'][i] = '체육'
        elif re.search("VMD", j) is not None:
            res_cer['certificate_contents'][i] = 'VMD'
        elif re.search("유통", j) is not None:
            res_cer['certificate_contents'][i] = '유통'
        elif re.search("조리", j) is not None:
            res_cer['certificate_contents'][i] = '조리'
        elif re.search("무선", j) is not None:
            res_cer['certificate_contents'][i] = '무선'
        elif re.search("샵마스터", j) is not None:
            res_cer['certificate_contents'][i] = '샵마스터'
        elif re.search("SHOP", j) is not None:
            res_cer['certificate_contents'][i] = '샵마스터'
        elif re.search("전산", j) is not None:
            res_cer['certificate_contents'][i] = '전산'
        elif re.search("정비", j) is not None:
            res_cer['certificate_contents'][i] = '정비'
        elif re.search("건축", j) is not None:
            res_cer['certificate_contents'][i] = '건축'
        elif re.search("양장", j) is not None:
            res_cer['certificate_contents'][i] = '양장'
        elif re.search("상담", j) is not None:
            res_cer['certificate_contents'][i] = '상담'
        elif re.search("무역", j) is not None:
            res_cer['certificate_contents'][i] = '무역'
        elif re.search("회계", j) is not None:
            res_cer['certificate_contents'][i] = '회계'
        elif re.search("MD", j) is not None:
            res_cer['certificate_contents'][i] = 'VMD'
        elif re.search("VISUAL", j) is not None:
            res_cer['certificate_contents'][i] = 'VMD'        
        elif re.search("매경", j) is not None:
            res_cer['certificate_contents'][i] = '매경' 
        elif re.search("한경", j) is not None:
            res_cer['certificate_contents'][i] = '매경' 
        elif re.search("금융", j) is not None:
            res_cer['certificate_contents'][i] = '금융'
        elif re.search("증권", j) is not None:
            res_cer['certificate_contents'][i] = '금융' 
        elif re.search("정비", j) is not None:
            res_cer['certificate_contents'][i] = '정비'
        elif re.search("원동기", j) is not None:
            res_cer['certificate_contents'][i] = '2종소형면허'            
        elif re.search("면허", j) is not None:
            res_cer['certificate_contents'][i] = '2종운전면허'
        elif re.search("컴퓨터", j) is not None:
            res_cer['certificate_contents'][i] = '컴활'
        elif re.search("파워포인트", j) is not None:
            res_cer['certificate_contents'][i] = '파워포인트'
        elif re.search("엑서스", j) is not None:
            res_cer['certificate_contents'][i] = '엑세스'
        elif re.search("디자인", j) is not None:
            res_cer['certificate_contents'][i] = '디자인'
        elif re.search("머천", j) is not None:
            res_cer['certificate_contents'][i] = 'VMD'
        elif re.search("기기", j) is not None:
            res_cer['certificate_contents'][i] = '기기'
        elif re.search("글", j) is not None:
            res_cer['certificate_contents'][i] = '글'
        elif re.search("머천", j) is not None:
            res_cer['certificate_contents'][i] = 'VMD'
        elif re.search("메이크업", j) is not None:
            res_cer['certificate_contents'][i] = '메이크업'
        elif re.search("제빵", j) is not None:
            res_cer['certificate_contents'][i] = '제빵'
        elif re.search("광고", j) is not None:
            res_cer['certificate_contents'][i] = '광고'
        elif re.search("바리스타", j) is not None:
            res_cer['certificate_contents'][i] = '바리스타'   
        elif re.search("텔레", j) is not None:
            res_cer['certificate_contents'][i] = '텔레마케팅'     
        elif re.search("마케팅", j) is not None:
            res_cer['certificate_contents'][i] = '마케팅'   
        elif re.search("마사지", j) is not None:
            res_cer['certificate_contents'][i] = '마사지'  
        elif re.search("스포츠", j) is not None:
            res_cer['certificate_contents'][i] = '스포츠'    
        elif re.search("아동", j) is not None:
            res_cer['certificate_contents'][i] = '아동'                       

In [145]:
res_cer['certificate_contents'].nunique()

740

In [100]:
res_cer.to_csv('keyword5.csv',encoding='EUC-KR',index=False)

PermissionError: [Errno 13] Permission denied: 'keyword5.csv'

In [53]:
from konlpy.tag import Okt   

In [40]:
okt = Okt()  
print(okt.nouns(res_cer['certificate_contents'][22]))   

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
okt = Okt()  

In [3]:
grouped = res_cer.groupby('resume_seq')['certificate_contents'].apply(list).reset_index()
grouped 

,resume_seq,certificate_contents
0,U00002,[자동차 운전면허 (2종 보통)]
1,U00003,"[컬러리스트 기사, 운전면허2종보통, 워드프로세서 1급]"
2,U00004,"[유통관리사 2급, 정보처리기능사, MOS EXCEL 2003 EXPERT]"
3,U00006,"[유통관리사 2급, 운전면허 1종 보통, 문서실무사1급]"
4,U00007,"[패턴 양장 기능사, 샵마스터, 자동차운전면허1종 보통, 스타일리스트, MICROS..."
...,...,...
5971,U08476,[운전면허2종보통]
5972,U08479,[컴퓨터그래픽스운용기능사]
5973,U08480,"[컴퓨터그래픽스운용기능사, 2종 보통 운전면허증, 컬러리스트기사, 컴퓨터패션디자인운..."
5974,U08481,[컴퓨터활용능력 2급]


In [69]:
merge_set1 = grouped.set_index(['resume_seq'])
merge_set1

,certificate_contents
resume_seq,
U00002,[자동차 운전면허 (2종 보통)]
U00003,"[컬러리스트 기사, 운전면허2종보통, 워드프로세서 1급]"
U00004,"[유통관리사 2급, 정보처리기능사, MOS EXCEL 2003 EXPERT]"
U00006,"[유통관리사 2급, 운전면허 1종 보통, 문서실무사1급]"
U00007,"[패턴 양장 기능사, 샵마스터, 자동차운전면허1종 보통, 스타일리스트, MICROS..."
...,...
U08476,[운전면허2종보통]
U08479,[컴퓨터그래픽스운용기능사]
U08480,"[컴퓨터그래픽스운용기능사, 2종 보통 운전면허증, 컬러리스트기사, 컴퓨터패션디자인운..."


In [70]:
res_edu = pd.read_csv("resume_education.csv")
eduGrouped = res_edu.sort_values('resume_seq')
len(eduGrouped['resume_seq'].unique()) 
# 학벌은 중복이 되지 않음.

8482

In [71]:
# 학벌은 중복이 되지 않음.
merge_set2 =eduGrouped.set_index(['resume_seq'])

In [72]:
res_lang= pd.read_csv("resume_language.csv")
langGrouped = res_lang.sort_values('resume_seq')
langGrouped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 869 entries, 45 to 707
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   resume_seq  869 non-null    object 
 1   language    869 non-null    int64  
 2   exam_name   869 non-null    int64  
 3   score       869 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 33.9+ KB


In [73]:
langGrouped['resume_seq'].value_counts()
# 언어 3개 시험을 본 사람이 있어서 묶을 필요가 존재함. 

U03660    3
U02601    3
U02424    2
U03578    2
U02691    2
         ..
U02950    1
U02963    1
U02972    1
U02999    1
U08482    1
Name: resume_seq, Length: 820, dtype: int64

In [74]:
langGrouped2 = langGrouped.groupby('resume_seq').agg({
    'language': list,
    'exam_name': list,
    'score': list
}).reset_index()
merge_set3 = langGrouped2.set_index(['resume_seq'])

In [75]:
res

,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,재료·화학·섬유·의복,NaN,NaN,67,NaN
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,재료·화학·섬유·의복,NaN,NaN,84,섬유;봉제;가방;의류
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,니트디자인,재료·화학·섬유·의복,NaN,NaN,121,학교;학원;직원훈련(교육서비스)
3,U04599,2020-05-13,2020-05-28,4,2012,0.0,2500.0,MD;기획MD,재료·화학·섬유·의복,NaN,NaN,24,섬유;봉제;가방;의류
4,U07573,2019-07-23,2020-03-08,4,2010,1900.0,0.0,디자이너;남성복;스포츠웨어;편집디자인;코디네이터;일러스트레이터;VMD;MD,재료·화학·섬유·의복,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U04890,2018-05-27,2020-08-26,5,1994,0.0,6250.0,상품기획;생산;글로벌소싱;남성복;여성복,재료·화학·섬유·의복,NaN,NaN,204,섬유;봉제;가방;의류
8478,U05910,2019-09-27,2020-10-22,4,2013,2300.0,1900.0,소재;기획MD;남성복;온라인MD;디자이너,재료·화학·섬유·의복,NaN,NaN,12,섬유;봉제;가방;의류
8479,U01468,2019-02-26,2020-06-23,4,2008,0.0,0.0,상품기획MD;우븐디자인,디자인,NaN,NaN,0,섬유;봉제;가방;의류
8480,U05315,2020-03-27,2020-12-21,4,2016,0.0,0.0,여성복디자인;우븐디자인;다이마루디자인,재료·화학·섬유·의복,NaN,NaN,18,디자인;CAD;설계


In [76]:
res_gro = pd.merge(res,grouped, left_on= 'resume_seq',right_on= 'resume_seq', how ='outer')
res_gro

,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code,certificate_contents
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,재료·화학·섬유·의복,NaN,NaN,67,NaN,NaN
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,재료·화학·섬유·의복,NaN,NaN,84,섬유;봉제;가방;의류,NaN
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,니트디자인,재료·화학·섬유·의복,NaN,NaN,121,학교;학원;직원훈련(교육서비스),[중등 2급 정교사]
3,U04599,2020-05-13,2020-05-28,4,2012,0.0,2500.0,MD;기획MD,재료·화학·섬유·의복,NaN,NaN,24,섬유;봉제;가방;의류,"[MCAS MASTER 2007, VMD MASTER]"
4,U07573,2019-07-23,2020-03-08,4,2010,1900.0,0.0,디자이너;남성복;스포츠웨어;편집디자인;코디네이터;일러스트레이터;VMD;MD,재료·화학·섬유·의복,NaN,NaN,0,NaN,"[컴퓨터그래픽스운용기능사, 워드프로세서 2급]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U04890,2018-05-27,2020-08-26,5,1994,0.0,6250.0,상품기획;생산;글로벌소싱;남성복;여성복,재료·화학·섬유·의복,NaN,NaN,204,섬유;봉제;가방;의류,NaN
8478,U05910,2019-09-27,2020-10-22,4,2013,2300.0,1900.0,소재;기획MD;남성복;온라인MD;디자이너,재료·화학·섬유·의복,NaN,NaN,12,섬유;봉제;가방;의류,"[엑서스), 워드, 파워포인트, 샵마스터 3급, MOS(엑셀]"
8479,U01468,2019-02-26,2020-06-23,4,2008,0.0,0.0,상품기획MD;우븐디자인,디자인,NaN,NaN,0,섬유;봉제;가방;의류,[자동차 운전면허증 1종보통]
8480,U05315,2020-03-27,2020-12-21,4,2016,0.0,0.0,여성복디자인;우븐디자인;다이마루디자인,재료·화학·섬유·의복,NaN,NaN,18,디자인;CAD;설계,[1종보통 운전면허증]


In [77]:
res_edu = pd.merge(res_gro,eduGrouped, left_on= 'resume_seq',right_on= 'resume_seq', how ='outer')
res_edu

,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,...,hischool_gender,hischool_location_seq,univ_type_seq1,univ_type_seq2,univ_transfer,univ_location,univ_major,univ_sub_major,univ_major_type,univ_score
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,재료·화학·섬유·의복,NaN,...,남여공학,11,5,5,0,11,NaN,NaN,4,70.0
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,재료·화학·섬유·의복,NaN,...,기타,0,5,5,0,5,NaN,NaN,4,90.0
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,니트디자인,재료·화학·섬유·의복,NaN,...,기타,0,5,5,0,3,NaN,NaN,9,90.0
3,U04599,2020-05-13,2020-05-28,4,2012,0.0,2500.0,MD;기획MD,재료·화학·섬유·의복,NaN,...,여자학교,7,3,5,0,11,NaN,NaN,19,80.0
4,U07573,2019-07-23,2020-03-08,4,2010,1900.0,0.0,디자이너;남성복;스포츠웨어;편집디자인;코디네이터;일러스트레이터;VMD;MD,재료·화학·섬유·의복,NaN,...,여자학교,4,5,5,0,13,NaN,NaN,19,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U04890,2018-05-27,2020-08-26,5,1994,0.0,6250.0,상품기획;생산;글로벌소싱;남성복;여성복,재료·화학·섬유·의복,NaN,...,남자학교,3,5,5,0,3,NaN,NaN,3,70.0
8478,U05910,2019-09-27,2020-10-22,4,2013,2300.0,1900.0,소재;기획MD;남성복;온라인MD;디자이너,재료·화학·섬유·의복,NaN,...,여자학교,15,5,5,0,15,NaN,NaN,19,80.0
8479,U01468,2019-02-26,2020-06-23,4,2008,0.0,0.0,상품기획MD;우븐디자인,디자인,NaN,...,기타,0,5,5,0,18,NaN,NaN,9,80.0
8480,U05315,2020-03-27,2020-12-21,4,2016,0.0,0.0,여성복디자인;우븐디자인;다이마루디자인,재료·화학·섬유·의복,NaN,...,남여공학,5,5,5,0,3,NaN,NaN,9,80.0


In [78]:
res_edu_lang = pd.merge(res_edu,langGrouped2 , left_on= 'resume_seq',right_on= 'resume_seq', how ='outer')
res_edu_lang

,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,...,univ_type_seq2,univ_transfer,univ_location,univ_major,univ_sub_major,univ_major_type,univ_score,language,exam_name,score
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,재료·화학·섬유·의복,NaN,...,5,0,11,NaN,NaN,4,70.0,NaN,NaN,NaN
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,재료·화학·섬유·의복,NaN,...,5,0,5,NaN,NaN,4,90.0,NaN,NaN,NaN
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,니트디자인,재료·화학·섬유·의복,NaN,...,5,0,3,NaN,NaN,9,90.0,NaN,NaN,NaN
3,U04599,2020-05-13,2020-05-28,4,2012,0.0,2500.0,MD;기획MD,재료·화학·섬유·의복,NaN,...,5,0,11,NaN,NaN,19,80.0,[2],[4],[792.93]
4,U07573,2019-07-23,2020-03-08,4,2010,1900.0,0.0,디자이너;남성복;스포츠웨어;편집디자인;코디네이터;일러스트레이터;VMD;MD,재료·화학·섬유·의복,NaN,...,5,0,13,NaN,NaN,19,80.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U04890,2018-05-27,2020-08-26,5,1994,0.0,6250.0,상품기획;생산;글로벌소싱;남성복;여성복,재료·화학·섬유·의복,NaN,...,5,0,3,NaN,NaN,3,70.0,NaN,NaN,NaN
8478,U05910,2019-09-27,2020-10-22,4,2013,2300.0,1900.0,소재;기획MD;남성복;온라인MD;디자이너,재료·화학·섬유·의복,NaN,...,5,0,15,NaN,NaN,19,80.0,NaN,NaN,NaN
8479,U01468,2019-02-26,2020-06-23,4,2008,0.0,0.0,상품기획MD;우븐디자인,디자인,NaN,...,5,0,18,NaN,NaN,9,80.0,NaN,NaN,NaN
8480,U05315,2020-03-27,2020-12-21,4,2016,0.0,0.0,여성복디자인;우븐디자인;다이마루디자인,재료·화학·섬유·의복,NaN,...,5,0,3,NaN,NaN,9,80.0,NaN,NaN,NaN


In [79]:
res_edu_lang.to_csv('total_resume.csv',encoding='EUC-KR',index=False)

In [80]:
res['job_code_seq1'].value_counts()

재료·화학·섬유·의복    7288
디자인             536
경영·기획·회계·사무     489
영업·판매·TM        158
건설·기계·전기·전자       5
전문·교육·자격          3
기타 직종             1
IT·게임             1
문화·예술·신문·방송       1
Name: job_code_seq1, dtype: int64

In [81]:
res['job_code_seq2'].value_counts()

영업·판매·TM       261
재료·화학·섬유·의복    132
디자인             54
전문·교육·자격        16
건설·기계·전기·전자      9
IT·게임            8
문화·예술·신문·방송      8
경영·기획·회계·사무      7
기술·과학·산업         3
여행사              1
기타 직종            1
Name: job_code_seq2, dtype: int64

In [82]:
res['job_code_seq3'].value_counts()

영업·판매·TM       73
전문·교육·자격       38
디자인            38
IT·게임          10
기타 직종          10
재료·화학·섬유·의복     6
기술·과학·산업        4
문화·예술·신문·방송     4
건설·기계·전기·전자     2
통신·모바일          1
관광              1
Name: job_code_seq3, dtype: int64

In [83]:
rec['address_seq1'].value_counts()

3.0     6490
5.0      174
20.0      17
11.0       5
1.0        4
13.0       2
9.0        1
2.0        1
Name: address_seq1, dtype: int64

In [84]:
rec['address_seq2'].value_counts()

5.0     35
1.0     29
20.0    24
11.0     4
3.0      2
13.0     1
2.0      1
7.0      1
8.0      1
9.0      1
17.0     1
Name: address_seq2, dtype: int64

In [85]:
rec['address_seq3'].value_counts()

20.0    6
3.0     1
8.0     1
18.0    1
Name: address_seq3, dtype: int64

In [86]:
res['career_job_code'].value_counts()

섬유;봉제;가방;의류            5059
도소매;유통;무역               821
디자인;CAD;설계              653
쇼핑몰;전자상거래;경매            133
기타 서비스                  119
학교;학원;직원훈련(교육서비스)        99
기타 제조                    81
광고;홍보                    81
영화;방송;공연;문화;스포츠          79
생활용품;화장품;완구              53
건설;토목;건축                 52
식음료;식품가공                 49
금융;보험;증권                 43
호텔;숙박;관광;여행;외식           35
전기;전자;통신장비               35
출판;인쇄;편집;영상              30
신문;서적;출판;인쇄              28
자동차;운송장비;조선;항공           27
공공행정;경찰;기관;협회            26
의료;제약;건강                 22
인터넷서비스;포털;컨텐츠            20
컴퓨터;사무기기;소비재             19
운송;운수;통신                 15
철강;금속;비금속;재료             15
반도체;LCD;광학;정밀            13
화학;고무;플라스틱               13
목재;나무;종이;가구              11
네트워크;통신;텔레콤              11
기타 공공;개인서비스              10
병원;제약;사회복지               10
기계;각종 장비                  9
웹에이전시                     9
인력공급;시설유지;TM(용역;도급)       8
IT 컨설팅;조사분석;통계            8
게임;캐릭터;애니메이션              7
전기;가스;수도;에너지        

In [87]:
rec_critical = rec[['recruitment_seq','text_keyword']]

In [88]:
res_critical = res[['resume_seq','text_keyword']]

In [89]:
ap = pd.read_csv('apply_train.csv')
ap = ap.sort_values('resume_seq')
ap

,resume_seq,recruitment_seq
11989,U00001,R05210
342,U00001,R05288
51174,U00001,R04536
28858,U00001,R06065
50591,U00002,R01103
...,...,...
13950,U08481,R00225
22895,U08481,R05500
33843,U08482,R03633
54343,U08482,R03743


In [90]:
crit1=pd.merge(ap,res_critical  , left_on= 'resume_seq',right_on= 'resume_seq', how ='outer')
crit1

,resume_seq,recruitment_seq,text_keyword
0,U00001,R05210,디자이너
1,U00001,R05288,디자이너
2,U00001,R04536,디자이너
3,U00001,R06065,디자이너
4,U00002,R01103,디자이너
...,...,...,...
57941,U08481,R00225,상품기획;머천다이저;기획MD
57942,U08481,R05500,상품기획;머천다이저;기획MD
57943,U08482,R03633,상품기획;영업MD;머천다이저;기획MD;마케팅;PR
57944,U08482,R03743,상품기획;영업MD;머천다이저;기획MD;마케팅;PR


In [91]:
crit2=pd.merge(crit1,rec_critical  , left_on= 'recruitment_seq',right_on= 'recruitment_seq', how ='outer')
crit2

,resume_seq,recruitment_seq,text_keyword_x,text_keyword_y
0,U00001,R05210,디자이너,NaN
1,U00803,R05210,NaN,NaN
2,U02861,R05210,머천다이저;디자이너,NaN
3,U05173,R05210,의류디자이너;의상디자이너;디스플레이;머천다이져;MD;VMD,NaN
4,U05680,R05210,일본무역;디자이너,NaN
...,...,...,...,...
57941,U07860,R04734,해외영업,NaN
57942,U07255,R00118,상품기획;MD;머천다이저,NaN
57943,U07661,R02219,해외영업;MD;상품기획,NaN
57944,U07821,R02219,MD,NaN


In [92]:
crit2.to_csv('keyword.csv',encoding='EUC-KR',index=False)

In [93]:
crit2['text_keyword_x']= crit2['text_keyword_x'].str.split(';')
crit2

,resume_seq,recruitment_seq,text_keyword_x,text_keyword_y
0,U00001,R05210,[디자이너],NaN
1,U00803,R05210,NaN,NaN
2,U02861,R05210,"[머천다이저, 디자이너]",NaN
3,U05173,R05210,"[의류디자이너, 의상디자이너, 디스플레이, 머천다이져, MD, VMD]",NaN
4,U05680,R05210,"[일본무역, 디자이너]",NaN
...,...,...,...,...
57941,U07860,R04734,[해외영업],NaN
57942,U07255,R00118,"[상품기획, MD, 머천다이저]",NaN
57943,U07661,R02219,"[해외영업, MD, 상품기획]",NaN
57944,U07821,R02219,[MD],NaN


In [94]:
pd.DataFrame(gl['text_keyword_x'].sum())

,text_keyword_x
recruitment_seq,
R00001,0.0
R00002,0.0
R00003,0.0
R00004,0.0
R00005,0.0
...,...
R06691,0.0
R06692,0.0
R06693,0.0


In [95]:
gl = crit2.groupby(['recruitment_seq'])
rec_list1 = list(pd.DataFrame(list(gl))[0])
len(rec_list1)


C:\Users\user\AppData\Local\Temp\ipykernel_3004\4064798178.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  rec_list1 = list(pd.DataFrame(list(gl))[0])


6695

In [96]:
gl['text_keyword_y'].sum()[2]

0

In [98]:
df_arr=[[]for i in range(len(rec_list1))] # 효율적이게 살자. 유지보수 귀찮잖아. 일일이 기록하지 말고 컴퓨터한테 이렇게 시키자. 
for i in range(len(rec_list1)):
    df_arr[i].append(rec_list1[i])
    count_df = pd.DataFrame(gl['text_keyword_x'].sum()[i])
    count_df_result = pd.DataFrame(count_df.value_counts()[count_df.value_counts() > 1]) 
    listy = []
    for j in range(len(count_df_result)): # 작업값이 있으면 가동한다.        
        listy.append(count_df_result.index[j][0]) # 핵심키워드를 하나하나 추출하여 리스트에 담는다. 
    df_arr[i].append(listy) # 리스트를 회사코드를 키로 설정하여 딕셔너리에 추가한다.     

harf_comp = pd.DataFrame(df_arr, columns= ['recruitment_seq','text_keyword_y'])
harf_comp

,recruitment_seq,text_keyword_y
0,R00001,"[거래처관리, 영업기획, 영업지원, 매출관리, 대리점관리, 매장관리, 매장영업관리,..."
1,R00002,[]
2,R00003,"[MD, VMD, 상품기획, 디스플레이, 마케팅, 머천다이저, 홍보]"
3,R00004,"[상품기획, VMD, 해외마케팅, 디스플레이, 디자이너, 패션마케팅, 머천다이저]"
4,R00005,[디자이너]
...,...,...
6690,R06691,"[여성복디자이너, 다이마루디자이너, 디자이너, 우븐디자이너]"
6691,R06692,"[영업관리, 생산관리, 영업기획, 기획, 생산, 마케팅기획, 마케팅]"
6692,R06693,[영업기획]
6693,R06694,"[의류, 상품기획, 해외영업, 머천다이징, 영업MD]"


In [99]:
for i in range(len(rec_list1)): # 세미콜론으로 전환하여 저장함. 
    harf_comp.iloc[i]['text_keyword_y']= ';'.join(harf_comp.iloc[i]['text_keyword_y'])
harf_comp

,recruitment_seq,text_keyword_y
0,R00001,거래처관리;영업기획;영업지원;매출관리;대리점관리;매장관리;매장영업관리;영업관리;백화...
1,R00002,
2,R00003,MD;VMD;상품기획;디스플레이;마케팅;머천다이저;홍보
3,R00004,상품기획;VMD;해외마케팅;디스플레이;디자이너;패션마케팅;머천다이저
4,R00005,디자이너
...,...,...
6690,R06691,여성복디자이너;다이마루디자이너;디자이너;우븐디자이너
6691,R06692,영업관리;생산관리;영업기획;기획;생산;마케팅기획;마케팅
6692,R06693,영업기획
6693,R06694,의류;상품기획;해외영업;머천다이징;영업MD


In [109]:
new_rec = rec.copy()
new_rec = new_rec.sort_values('recruitment_seq')
new_rec

,recruitment_seq,address_seq1,address_seq2,address_seq3,career_end,career_start,check_box_keyword,education,major_task,qualifications,text_keyword
4573,R00001,5.0,NaN,NaN,0,0,2101;2108;2201;2204;2205;2707;2810,2,2,1,NaN
5915,R00002,3.0,NaN,NaN,0,0,2507;2703;2707,3,2,1,NaN
6454,R00003,3.0,NaN,NaN,0,0,2101;2108;2201;2707,3,2,2,NaN
5905,R00004,3.0,NaN,NaN,0,0,2507;2707,3,2,1,NaN
4197,R00005,3.0,NaN,NaN,0,0,2507;2707,3,2,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5983,R06691,3.0,NaN,NaN,0,0,2501;2507;2707,3,2,1,NaN
2367,R06692,3.0,NaN,NaN,0,0,2201;2507,3,2,2,NaN
6235,R06693,5.0,NaN,NaN,0,0,2102;2707,4,2,1,NaN
3298,R06694,3.0,NaN,NaN,0,0,2101;2108;2109;2110;2201;2203;2707,4,2,1,NaN


In [34]:
for i in range(len(rec_list1)):
    new_rec.loc['recruitment_seq'] = rec_list1#, 'text_keyword_y'] = rec_dic[main_name]
crit2

,resume_seq,recruitment_seq,text_keyword_x,text_keyword_y
0,U00001,R05210,[디자이너],거래처관리;영업기획;영업지원;매출관리;대리점관리;매장관리;매장영업관리;영업관리;백화...
1,U00803,R05210,NaN,거래처관리;영업기획;영업지원;매출관리;대리점관리;매장관리;매장영업관리;영업관리;백화...
2,U02861,R05210,"[머천다이저, 디자이너]",거래처관리;영업기획;영업지원;매출관리;대리점관리;매장관리;매장영업관리;영업관리;백화...
3,U05173,R05210,"[의류디자이너, 의상디자이너, 디스플레이, 머천다이져, MD, VMD]",거래처관리;영업기획;영업지원;매출관리;대리점관리;매장관리;매장영업관리;영업관리;백화...
4,U05680,R05210,"[일본무역, 디자이너]",거래처관리;영업기획;영업지원;매출관리;대리점관리;매장관리;매장영업관리;영업관리;백화...
...,...,...,...,...
57941,U07860,R04734,[해외영업],영업기획
57942,U07255,R00118,"[상품기획, MD, 머천다이저]",의류;상품기획;해외영업;머천다이징;영업MD
57943,U07661,R02219,"[해외영업, MD, 상품기획]",디자이너;여성복디자이너;여성복디자인
57944,U07821,R02219,[MD],디자이너;여성복디자이너;여성복디자인


In [113]:
final1 = pd.merge(new_rec,harf_comp , left_on= 'recruitment_seq',right_on= 'recruitment_seq', how ='outer')
final1.to_csv('keyword4.csv',encoding='EUC-KR',index=False)

In [ ]:
bbbb

In [121]:
res3 = res.sort_values('resume_seq')
res3
deg_dum= pd.get_dummies(res3['degree']).add_prefix("degree_")

In [133]:
final2 = pd.merge(res3,deg_dum , left_index=True ,right_index=True , how ='outer')
final2 = final2.drop(columns=['degree'])
final2

,resume_seq,reg_date,updated_date,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code,degree_2,degree_3,degree_4,degree_5,degree_6
1690,U00001,2019-12-27,2020-02-01,2009,1500.0,1500.0,디자이너,재료·화학·섬유·의복,NaN,NaN,0,NaN,0,0,1,0,0
3979,U00002,2020-04-24,2020-04-29,2005,0.0,0.0,디자이너,재료·화학·섬유·의복,NaN,NaN,73,기타 공공;개인서비스,0,0,1,0,0
5648,U00003,2018-02-14,2020-07-08,2004,0.0,0.0,남성복디자이너;TD캐주얼,재료·화학·섬유·의복,NaN,NaN,113,섬유;봉제;가방;의류,0,0,1,0,0
1367,U00004,2017-10-26,2020-04-27,2011,0.0,0.0,상품기획;영업기획,재료·화학·섬유·의복,NaN,NaN,27,섬유;봉제;가방;의류,0,0,1,0,0
5242,U00005,2020-03-18,2020-04-08,2011,0.0,0.0,인사;총무;경영;MD;상품기획,재료·화학·섬유·의복,NaN,NaN,0,NaN,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,U08478,2020-02-05,2020-03-31,2003,0.0,2700.0,상품기획,재료·화학·섬유·의복,NaN,NaN,60,섬유;봉제;가방;의류,0,0,1,0,0
2227,U08479,2016-02-17,2020-07-15,0,2500.0,2300.0,디자이너,재료·화학·섬유·의복,NaN,NaN,59,디자인;CAD;설계,0,1,0,0,0
2657,U08480,2019-11-13,2020-06-24,0,0.0,0.0,VMD;텍스타일디자인;섬유디자인;니트디자인;여성복디자인,디자인,NaN,NaN,0,NaN,0,0,1,0,0
6619,U08481,2020-07-09,2020-11-23,2015,0.0,2700.0,상품기획;머천다이저;기획MD,재료·화학·섬유·의복,NaN,NaN,8,섬유;봉제;가방;의류,0,0,1,0,0


In [131]:
res['job_code_seq1'] 

재료·화학·섬유·의복    7288
디자인             536
경영·기획·회계·사무     489
영업·판매·TM        158
건설·기계·전기·전자       5
전문·교육·자격          3
기타 직종             1
IT·게임             1
문화·예술·신문·방송       1
Name: job_code_seq1, dtype: int64

In [135]:
deg_dum2 = pd.get_dummies(res['job_code_seq1']).add_prefix("job_")

In [136]:
final3 = pd.merge(final2,deg_dum2 , left_index=True ,right_index=True , how ='outer')


,resume_seq,reg_date,updated_date,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,...,degree_6,job_IT·게임,job_건설·기계·전기·전자,job_경영·기획·회계·사무,job_기타 직종,job_디자인,job_문화·예술·신문·방송,job_영업·판매·TM,job_재료·화학·섬유·의복,job_전문·교육·자격
0,U00606,2020-03-04,2020-05-22,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,재료·화학·섬유·의복,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
1,U00509,2019-08-25,2020-09-02,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,재료·화학·섬유·의복,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
2,U02012,2017-11-20,2020-01-26,1979,3500.0,3100.0,니트디자인,재료·화학·섬유·의복,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
3,U04599,2020-05-13,2020-05-28,2012,0.0,2500.0,MD;기획MD,재료·화학·섬유·의복,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
4,U07573,2019-07-23,2020-03-08,2010,1900.0,0.0,디자이너;남성복;스포츠웨어;편집디자인;코디네이터;일러스트레이터;VMD;MD,재료·화학·섬유·의복,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U04890,2018-05-27,2020-08-26,1994,0.0,6250.0,상품기획;생산;글로벌소싱;남성복;여성복,재료·화학·섬유·의복,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
8478,U05910,2019-09-27,2020-10-22,2013,2300.0,1900.0,소재;기획MD;남성복;온라인MD;디자이너,재료·화학·섬유·의복,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
8479,U01468,2019-02-26,2020-06-23,2008,0.0,0.0,상품기획MD;우븐디자인,디자인,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
8480,U05315,2020-03-27,2020-12-21,2016,0.0,0.0,여성복디자인;우븐디자인;다이마루디자인,재료·화학·섬유·의복,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0


In [142]:
final3 = final3.drop(columns=['job_code_seq1','job_code_seq2','job_code_seq3'])
final3

In [146]:
final3['graduate_date'][final3['graduate_date'] == 0]

1       0
6       0
24      0
25      0
29      0
       ..
8443    0
8445    0
8449    0
8454    0
8456    0
Name: graduate_date, Length: 792, dtype: int64

In [144]:
(final3['graduate_date'] -min(final3['graduate_date'])) / (max(final3['graduate_date'])-min(final3['graduate_date']))

0       0.993076
1       0.000000
2       0.978734
3       0.995054
4       0.994065
          ...   
8477    0.986152
8478    0.995549
8479    0.993076
8480    0.997033
8481    0.993571
Name: graduate_date, Length: 8482, dtype: float64

In [163]:
for i in range(len(final3['graduate_date'])):
    if final3['graduate_date'].loc[i] == 0:
        final3['graduate_date'].loc[i] = 2020 - final3['career_month'].loc[i]/6

C:\Users\user\AppData\Local\Temp\ipykernel_3004\2523133591.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final3['graduate_date'].loc[i] = 2020 - final3['career_month'].loc[i]/6


In [166]:
final3['graduate_date'] = (final3['graduate_date'] -min(final3['graduate_date'])) / (max(final3['graduate_date'])-min(final3['graduate_date']))

In [168]:
final3['career_month'] = (final3['career_month'] -min(final3['career_month'])) / (max(final3['career_month'])-min(final3['career_month']))
final3['career_month'] 

0       0.155093
1       0.194444
2       0.280093
3       0.055556
4       0.000000
          ...   
8477    0.472222
8478    0.027778
8479    0.000000
8480    0.041667
8481    0.006944
Name: career_month, Length: 8482, dtype: float64

In [169]:
final3.columns

Index(['resume_seq', 'reg_date', 'updated_date', 'graduate_date',
       'hope_salary', 'last_salary', 'text_keyword', 'career_month',
       'career_job_code', 'degree_2', 'degree_3', 'degree_4', 'degree_5',
       'degree_6', 'job_IT·게임', 'job_건설·기계·전기·전자', 'job_경영·기획·회계·사무',
       'job_기타 직종', 'job_디자인', 'job_문화·예술·신문·방송', 'job_영업·판매·TM',
       'job_재료·화학·섬유·의복', 'job_전문·교육·자격'],
      dtype='object')

In [170]:
from sklearn.metrics.pairwise import cosine_similarity

In [185]:
final4 = final3[['resume_seq', 'graduate_date','career_month', 'degree_2', 'degree_3', 'degree_4', 'degree_5',
       'degree_6', 'job_IT·게임', 'job_건설·기계·전기·전자', 'job_경영·기획·회계·사무',
       'job_기타 직종', 'job_디자인', 'job_문화·예술·신문·방송', 'job_영업·판매·TM',
       'job_재료·화학·섬유·의복', 'job_전문·교육·자격']]
final4 = final4.set_index(['resume_seq'])

In [180]:
from sklearn.linear_model import LogisticRegression

In [188]:
ans = cosine_similarity(final4)

In [189]:
ans.shape

(8482, 8482)

array([0.07248187, 0.09087334, 0.09547115, ..., 0.99999896, 1.        ,
       1.        ])

None
